## Inicializar la Dimensión Fecha
Importaciones

In [1]:
from datetime import date
import pandas as pd
import numpy as np
import holidays
import yaml

# Añadir los atributos básicos como año, mes, día, trimestre, y semana del año.

In [7]:
# Crear un rango de fechas por minuto para obtener horas y minutos también
dates = pd.date_range(start='1/1/2023', end='12/31/2025 23:59', freq='min')

# Inicializar el DataFrame
dim_fecha = pd.DataFrame({
    "fecha_completa": dates,  # La fecha completa en formato YYYY-MM-DD HH:MM
})

dim_fecha.head()

,fecha_completa
0,2023-01-01 00:00:00
1,2023-01-01 00:01:00
2,2023-01-01 00:02:00
3,2023-01-01 00:03:00
4,2023-01-01 00:04:00


# Añadir columnas para horas y minutos, y marcar si es día laboral, fin de semana o feriado.

In [8]:
# Añadir columnas como el año, mes, día, día de la semana y el trimestre del año.

dim_fecha["fecha_id"] = dim_fecha["fecha_completa"].dt.strftime('%Y%m%d').astype(int)  # ID de fecha en formato YYYYMMDD
dim_fecha["año"] = dim_fecha["fecha_completa"].dt.year
dim_fecha["mes"] = dim_fecha["fecha_completa"].dt.month
dim_fecha["nombre_mes"] = dim_fecha["fecha_completa"].dt.month_name()
dim_fecha["trimestre"] = dim_fecha["fecha_completa"].dt.quarter
dim_fecha["semana"] = dim_fecha["fecha_completa"].dt.isocalendar().week
dim_fecha["dia"] = dim_fecha["fecha_completa"].dt.day
dim_fecha["nombre_dia"] = dim_fecha["fecha_completa"].dt.day_name()

dim_fecha.head()


,fecha_completa,fecha_id,año,mes,nombre_mes,trimestre,semana,dia,nombre_dia
0,2023-01-01 00:00:00,20230101,2023,1,January,1,52,1,Sunday
1,2023-01-01 00:01:00,20230101,2023,1,January,1,52,1,Sunday
2,2023-01-01 00:02:00,20230101,2023,1,January,1,52,1,Sunday
3,2023-01-01 00:03:00,20230101,2023,1,January,1,52,1,Sunday
4,2023-01-01 00:04:00,20230101,2023,1,January,1,52,1,Sunday


# Añadir el periodo del día ('Mañana', 'Tarde', 'Noche') basado en la hora del día.


In [15]:
#Añadir columnas para horas y minutos, y marcar si es día laboral, fin de semana o feriado.
#Añadir la hora y el minuto
dim_fecha["hora"] = dim_fecha["fecha_completa"].dt.hour
dim_fecha["minuto"] = dim_fecha["fecha_completa"].dt.minute

In [5]:
# Añadir información sobre feriados y si es fin de semana.

co_holidays = holidays.CO(language="es")  # Usar librería holidays para identificar feriados en Colombia

# Aplicar las reglas para identificar si es feriado
dim_fecha["is_Holiday"] = dim_fecha["date"].apply(lambda x: x in co_holidays)
dim_fecha["holiday"] = dim_fecha["date"].apply(lambda x: co_holidays.get(x))
dim_fecha["saved"] = date.today()  # Añadir fecha de guardado
dim_fecha["weekend"] = dim_fecha["weekday"].apply(lambda x: x > 4)  # Identificar si es fin de semana (sábado o domingo)

dim_fecha.head()


,date,year,month,day,weekday,quarter,day_of_year,day_of_month,month_str,day_str,date_str,is_Holiday,holiday,saved,weekend
0,2023-01-01,2023,1,1,6,1,1,31,January,Sunday,01/01/2023,True,Año Nuevo,2024-10-14,True
1,2023-01-02,2023,1,2,0,1,2,31,January,Monday,02/01/2023,False,None,2024-10-14,False
2,2023-01-03,2023,1,3,1,1,3,31,January,Tuesday,03/01/2023,False,None,2024-10-14,False
3,2023-01-04,2023,1,4,2,1,4,31,January,Wednesday,04/01/2023,False,None,2024-10-14,False
4,2023-01-05,2023,1,5,3,1,5,31,January,Thursday,05/01/2023,False,None,2024-10-14,False


# # Conexión a la Base de Datos
Ahora vamos a conectarnos a la base de datos usando SQLAlchemy para cargar la dimensión fecha.

In [11]:
# Añadir indicadores booleanos para días laborales y fines de semana
dim_fecha["es_dia_laboral"] = dim_fecha["fecha_completa"].dt.weekday < 5  # True si es de lunes a viernes
dim_fecha["es_fin_de_semana"] = dim_fecha["fecha_completa"].dt.weekday >= 5  # True si es sábado o domingo

# Marcar feriados en Colombia
co_holidays = holidays.CO(years=dim_fecha["año"].unique(), language="es")
dim_fecha["es_feriado"] = dim_fecha["fecha_completa"].dt.date.isin(co_holidays)

dim_fecha.head()

,fecha_completa,fecha_id,año,mes,nombre_mes,trimestre,semana,dia,nombre_dia,hora,minuto,es_dia_laboral,es_fin_de_semana,es_feriado
0,2023-01-01 00:00:00,20230101,2023,1,January,1,52,1,Sunday,0,0,False,True,True
1,2023-01-01 00:01:00,20230101,2023,1,January,1,52,1,Sunday,0,1,False,True,True
2,2023-01-01 00:02:00,20230101,2023,1,January,1,52,1,Sunday,0,2,False,True,True
3,2023-01-01 00:03:00,20230101,2023,1,January,1,52,1,Sunday,0,3,False,True,True
4,2023-01-01 00:04:00,20230101,2023,1,January,1,52,1,Sunday,0,4,False,True,True


In [12]:
# Añadir el periodo del día ('Mañana', 'Tarde', 'Noche') basado en la hora del día.

# Definir el periodo del día basado en la hora
def asignar_periodo_dia(hora):
    if 0 <= hora < 12:
        return 'Mañana'
    elif 12 <= hora < 18:
        return 'Tarde'
    else:
        return 'Noche'


dim_fecha["periodo_dia"] = dim_fecha["hora"].apply(asignar_periodo_dia)

dim_fecha.head()


,fecha_completa,fecha_id,año,mes,nombre_mes,trimestre,semana,dia,nombre_dia,hora,minuto,es_dia_laboral,es_fin_de_semana,es_feriado,periodo_dia
0,2023-01-01 00:00:00,20230101,2023,1,January,1,52,1,Sunday,0,0,False,True,True,Mañana
1,2023-01-01 00:01:00,20230101,2023,1,January,1,52,1,Sunday,0,1,False,True,True,Mañana
2,2023-01-01 00:02:00,20230101,2023,1,January,1,52,1,Sunday,0,2,False,True,True,Mañana
3,2023-01-01 00:03:00,20230101,2023,1,January,1,52,1,Sunday,0,3,False,True,True,Mañana
4,2023-01-01 00:04:00,20230101,2023,1,January,1,52,1,Sunday,0,4,False,True,True,Mañana


In [18]:
# # Conexión a la Base de Datos
# Ahora vamos a conectarnos a la base de datos usando SQLAlchemy para cargar la dimensión fecha.

from sqlalchemy import create_engine
import yaml

# Cargar configuración de conexión desde el archivo YAML
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_co = config['RAPIDOS-Y_FURIOSOS']
    config_etl = config['ETL_RYF']

# Construir la URL de la base de datos
url_co = (f"{config_co['drivername']}://{config_co['user']}:{config_co['password']}@{config_co['host']}:"
          f"{config_co['port']}/{config_co['dbname']}")
url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@127.0.0.1:"
           f"{config_etl['port']}/{config_etl['dbname']}")


# Crear el motor SQLAlchemy para la conexión
co_sa = create_engine(url_co)
etl_conn = create_engine(url_etl)


# Cargar la Dimensión Fecha a la Base de Datos
Cargar el DataFrame `dim_fecha` a la tabla de la base de datos usando SQLAlchemy.

In [19]:
dim_fecha.to_sql('dim_fecha', etl_conn, if_exists='replace', index_label='key_dim_fecha')

OperationalError: (psycopg2.OperationalError) connection to server at "127.0.0.1", port 5432 failed: FATAL:  la autentificación password falló para el usuario «postgresql»

(Background on this error at: https://sqlalche.me/e/20/e3q8)